In [5]:
import pandas as pd
import numpy as np
import datetime
from Trax.Cloud.Services.Connector.Logger import LoggerInitializer
from Trax.Utils.Conf.Configuration import Config
from Trax.Data.Projects.Connector import ProjectConnector
from Trax.Cloud.Services.Connector.Keys import DbUsers

LoggerInitializer.init('Diageo Accuracy Test')
Config.set_access_mode(Config.CROSS_CLOUD_ACCESS_MODE)
#Config.set_env_and_cloud(Config.PROD, Config.AWS)
Config.set_env_and_cloud(Config.PROD, Config.GCP)

2021-11-23 17:12:47,469 - Diageo Accuracy Test - 43845 - WARNING - Logger init method is called more than once. Current app:Diageo Accuracy Test, new app: Diageo Accuracy Test 


False

In [6]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [7]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [8]:
from Trax.Cloud.Services.Connector.Factory import BigQueryFactory
try:
    print(bq_client)
except NameError:
    bq_client = BigQueryFactory.get_bigquery_client('trax-ortal-prod')

In [11]:
sql_probes = """ SELECT timestamp,event_name,session_uid,wave_type,wave_uid,
                        project_name,probe_id,data_subject,flavor,accuracy_logic_type,
                        fr.masking_box,fr.majority_product_fk,
                        fr.voting_result,fr.voting_result_linkage, fr.majority_value,
                        fr.majority_is_promotion, fr.production_tag_source_group,
                        fr.production_tag_source, fr.production_tag_identified,
                        fr.production_product_fk, fr.TP_value, fr.FP_value, fr.FN_value,
                        fr.INCONCLUSIVE_value, fr.TP_promotion,fr.FP_promotion,fr.FN_promotion,
                        fr.TP_linkage,fr.FP_linkage,fr.FN_linkage,fr.INCONCLUSIVE_linkage
                    FROM `trax-ortal-prod.raw.factory_accuracy_price`
                    join unnest (additional_event_data) fr
                    WHERE DATE(timestamp) > "2021-11-01" 
                    """
df = bq_client.run_query(sql_probes).to_dataframe()

2021-11-23 17:13:45,711 - Diageo Accuracy Test - 43845 - INFO - Run query with job_id=58ad0d90-887d-4aca-a920-4771930920a6 
2021-11-23 17:13:52,911 - Diageo Accuracy Test - 43845 - INFO - Query job=58ad0d90-887d-4aca-a920-4771930920a6 finished {'total_mb_billed': 214.95808, 'total_mb_processed': 214.873181}


In [12]:
df.head()

,timestamp,event_name,session_uid,wave_type,wave_uid,project_name,probe_id,data_subject,flavor,accuracy_logic_type,masking_box,majority_product_fk,voting_result,voting_result_linkage,majority_value,majority_is_promotion,production_tag_source_group,production_tag_source,production_tag_identified,production_product_fk,TP_value,FP_value,FN_value,INCONCLUSIVE_value,TP_promotion,FP_promotion,FN_promotion,TP_linkage,FP_linkage,FN_linkage,INCONCLUSIVE_linkage
0,2021-11-23 15:11:23.958166+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,476aca0e-3382-4849-8ee5-2115a4a9b4b1,pricing_voting,88554af8-fdc2-4dce-9f9a-f5e34297cdcc,cencosudpe,299817,Price,default,internal,1366777,NaN,majority,inconclusive,31.0,0.0,QAT,QAT,1,40649.0,1,0,0,0,1,0,0,0,0,0,1
1,2021-11-23 15:11:23.958166+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,476aca0e-3382-4849-8ee5-2115a4a9b4b1,pricing_voting,88554af8-fdc2-4dce-9f9a-f5e34297cdcc,cencosudpe,299817,Price,default,internal,1366780,NaN,majority,inconclusive,23.0,0.0,QAT,QAT,1,33426.0,1,0,0,0,1,0,0,0,0,0,1
2,2021-11-23 15:11:23.958166+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,476aca0e-3382-4849-8ee5-2115a4a9b4b1,pricing_voting,88554af8-fdc2-4dce-9f9a-f5e34297cdcc,cencosudpe,299817,Price,default,internal,1366781,NaN,majority,inconclusive,34.0,0.0,QAT,QAT,1,33429.0,1,0,0,0,1,0,0,0,0,0,1
3,2021-11-23 15:11:23.958166+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,476aca0e-3382-4849-8ee5-2115a4a9b4b1,pricing_voting,88554af8-fdc2-4dce-9f9a-f5e34297cdcc,cencosudpe,299817,Price,default,internal,1366778,NaN,majority,inconclusive,23.0,0.0,QAT,QAT,1,NaN,1,0,0,0,1,0,0,0,0,0,1
4,2021-11-23 15:11:23.958166+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,476aca0e-3382-4849-8ee5-2115a4a9b4b1,pricing_voting,88554af8-fdc2-4dce-9f9a-f5e34297cdcc,cencosudpe,299817,Price,default,internal,1366779,NaN,majority,inconclusive,23.0,0.0,QAT,QAT,1,NaN,1,0,0,0,1,0,0,0,0,0,1


In [14]:
df.accuracy_logic_type.unique()

array(['internal'], dtype=object)

In [15]:
df.data_subject.unique()

array(['Price'], dtype=object)

In [16]:
df.flavor.unique()

array(['default'], dtype=object)

In [17]:
t1 = df.groupby(['timestamp','project_name','probe_id','majority_product_fk']).agg({'voting_result':'count'}).reset_index()

In [19]:
t1[t1['voting_result']>1]

,timestamp,project_name,probe_id,majority_product_fk,voting_result
39,2021-11-02 01:04:41.745355+00:00,jnjanz,664187,5510.0,3
41,2021-11-02 01:04:41.745355+00:00,jnjanz,664187,6750.0,11
349,2021-11-03 05:46:28.618424+00:00,danonear,38552,349.0,2
420,2021-11-03 06:24:40.297582+00:00,danonear,40848,410.0,2
441,2021-11-03 06:27:56.353324+00:00,danonear,41332,365.0,2
455,2021-11-03 08:49:42.941671+00:00,diageobenelux,836946,498.0,14
456,2021-11-03 08:49:42.941671+00:00,diageobenelux,836946,524.0,6
458,2021-11-03 08:49:42.941671+00:00,diageobenelux,836946,631.0,2
462,2021-11-03 08:49:42.941671+00:00,diageobenelux,836946,2897.0,3
612,2021-11-04 00:57:56.722732+00:00,nestleus,10896076,67300.0,2


In [22]:
df[(df['probe_id']==664187) & (df['project_name']=='jnjanz')&(df['majority_product_fk']==5510.0)]

,timestamp,event_name,session_uid,wave_type,wave_uid,project_name,probe_id,data_subject,flavor,accuracy_logic_type,masking_box,majority_product_fk,voting_result,voting_result_linkage,majority_value,majority_is_promotion,production_tag_source_group,production_tag_source,production_tag_identified,production_product_fk,TP_value,FP_value,FN_value,INCONCLUSIVE_value,TP_promotion,FP_promotion,FN_promotion,TP_linkage,FP_linkage,FN_linkage,INCONCLUSIVE_linkage
1025562,2021-11-02 01:04:41.745355+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,9E042EDF-B2F5-4E8B-B00C-0233FECAD718,voting,39c24ecc-0632-43a6-869f-c96a1bcb5b5d,jnjanz,664187,Price,default,internal,3958139,5510.0,majority,majority,59.99,0.0,Engine,Engine,1,NaN,1,0,0,0,1,0,0,0,0,1,0
1025563,2021-11-02 01:04:41.745355+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,9E042EDF-B2F5-4E8B-B00C-0233FECAD718,voting,39c24ecc-0632-43a6-869f-c96a1bcb5b5d,jnjanz,664187,Price,default,internal,3958138,5510.0,majority,majority,59.99,0.0,Engine,Engine,1,NaN,1,0,0,0,1,0,0,0,0,1,0
1025564,2021-11-02 01:04:41.745355+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,9E042EDF-B2F5-4E8B-B00C-0233FECAD718,voting,39c24ecc-0632-43a6-869f-c96a1bcb5b5d,jnjanz,664187,Price,default,internal,3958137,5510.0,majority,majority,59.99,0.0,Engine,Engine,1,NaN,1,0,0,0,1,0,0,0,0,1,0


In [25]:
sql = """ select *
from trax-ortal-prod.thelake.rds_product
"""
rds_product = bq_client.run_query(sql).to_dataframe()

2021-11-23 18:22:28,924 - Diageo Accuracy Test - 43845 - INFO - Run query with job_id=92059b5b-3461-4c86-9272-7dd4bd3375d7 
2021-11-23 18:22:49,510 - Diageo Accuracy Test - 43845 - INFO - Query job=92059b5b-3461-4c86-9272-7dd4bd3375d7 finished {'total_mb_billed': 761.266176, 'total_mb_processed': 760.73206}


In [26]:
rds_product.head()

,product_pk,product_uuid,product_name,product_local_name,product_type,ean_code,item_code,form_factor,size,unit_of_measure,unit_num,subpackages_num,activation_update_time,creation_date,delete_date,product_width,product_height,product_depth,sku_global_code,is_traxlive,brand_pk,brand_name,brand_local_name,brand_creation_date,brand_delete_date,brand_uuid,brand_global_code,brand_recognition_level_fk,manufacture_name,is_project_manufacturer,brand_recognition_level,category_pk,category_name,category_local_name,category_uuid,trax_category_pk,trax_category_name,trax_category_segment,is_active,manufacturer_pk,trax_category_uuid,labels,substitution_product_fk,trax_palette_level_1_fk,trax_palette_level_1_value,trax_palette_level_2_fk,trax_palette_level_2_value,sub_category_pk,sub_category_name,project_name,_snapshot_date,cloud
0,9455,52b1cbc0-1e9b-11eb-bca5-837bf363b59c,Cookies & Cakes Other,Cookies & Cakes Other,Other,None,None,None,NaN,None,NaN,NaN,2020-11-04 12:43:30+00:00,2020-11-04 12:43:22+00:00,NaT,NaN,NaN,NaN,None,1,0,General,General,2018-04-08 13:31:21+00:00,NaT,00000000-0000-0000-0000-000000000000,None,1,General,True,Brand,0,General,General,00000000-0000-0000-0000-000000000000,27.0,Cookies & Cakes,Food Cupboard,1,0.0,3ee098a6-1dee-455e-b202-d9f32ec37b21,{},NaN,[73],[],[3487],"[""General""]",NaN,None,ccru,2021-11-23,AWS
1,2408,62b1dc27-4b7f-11ec-99da-0ae626d9f1e9,General Empty,General Empty,Empty,Empty0496,None,None,NaN,None,NaN,NaN,NaT,2017-12-17 08:40:06+00:00,2018-12-03 12:32:58+00:00,NaN,NaN,NaN,None,0,0,General,General,2018-04-08 13:31:21+00:00,NaT,00000000-0000-0000-0000-000000000000,None,1,General,True,Brand,0,General,General,00000000-0000-0000-0000-000000000000,None,None,None,0,0.0,None,{},NaN,[],[],[],[],NaN,None,ccru,2021-11-23,AWS
2,0,62b1db91-4b7f-11ec-99da-0ae626d9f1e9,General Empty,General Empty,Empty,None,None,None,NaN,None,NaN,NaN,NaT,2017-12-17 08:40:06+00:00,NaT,NaN,NaN,NaN,None,0,0,General,General,2018-04-08 13:31:21+00:00,NaT,00000000-0000-0000-0000-000000000000,None,1,General,True,Brand,0,General,General,00000000-0000-0000-0000-000000000000,None,None,None,0,0.0,None,None,NaN,[],[],[],[],NaN,None,ccru,2021-11-23,AWS
3,2282,b43bdbd0-cf17-11e9-8814-95f250360c3b,Irrelevant,Irrelevant,Irrelevant,None,None,None,NaN,None,NaN,NaN,2019-09-04 13:28:03+00:00,2019-09-04 13:27:12+00:00,NaT,NaN,NaN,NaN,None,0,0,General,General,2018-04-08 13:31:30+00:00,NaT,00000000-0000-0000-0000-000000000000,None,1,General,True,Brand,0,General,General,00000000-0000-0000-0000-000000000000,65.0,General,General,1,0.0,e2b80fe0-22c8-4c17-8067-c421a23ce301,"{""SEGMENTO"": ""Irrelevant"", ""CATEGORÍA"": ""Irrel...",NaN,[28],[],[372],"[""General""]",NaN,None,ccnayarmx,2021-11-23,GCP
4,66089,b288b820-b1cc-11e9-b3da-5d9ec8d99d23,Irrelevant Product,Irrelevant Product,Irrelevant,None,None,None,NaN,None,NaN,NaN,2019-12-31 20:54:28+00:00,2019-07-29 06:47:13+00:00,NaT,NaN,NaN,NaN,None,0,0,General,General,2018-04-08 13:31:30+00:00,NaT,00000000-0000-0000-0000-000000000000,None,1,General,True,Brand,0,General,General,00000000-0000-0000-0000-000000000000,65.0,General,General,1,0.0,e2b80fe0-22c8-4c17-8067-c421a23ce301,"{""att4"": ""GENERAL"", ""item_id"": ""3373""}",NaN,[108],[],[8351],"[""General""]",NaN,None,cclibertyus,2021-11-23,AWS


In [29]:
rds_product[(rds_product['project_name']=='jnjanz')&(rds_product['product_pk']==5510)]

,product_pk,product_uuid,product_name,product_local_name,product_type,ean_code,item_code,form_factor,size,unit_of_measure,unit_num,subpackages_num,activation_update_time,creation_date,delete_date,product_width,product_height,product_depth,sku_global_code,is_traxlive,brand_pk,brand_name,brand_local_name,brand_creation_date,brand_delete_date,brand_uuid,brand_global_code,brand_recognition_level_fk,manufacture_name,is_project_manufacturer,brand_recognition_level,category_pk,category_name,category_local_name,category_uuid,trax_category_pk,trax_category_name,trax_category_segment,is_active,manufacturer_pk,trax_category_uuid,labels,substitution_product_fk,trax_palette_level_1_fk,trax_palette_level_1_value,trax_palette_level_2_fk,trax_palette_level_2_value,sub_category_pk,sub_category_name,project_name,_snapshot_date,cloud
652464,5510,eeddff30-678c-11ea-ba7a-6b7d0b873a3d,Olay Face / Body Care Other,Olay Face / Body Care Other,Other,EAN221,None,None,NaN,None,NaN,NaN,2020-03-16 14:32:56+00:00,2020-03-16 13:49:18+00:00,NaT,NaN,NaN,NaN,None,0,762,OLAY,OLAY,2020-03-16 13:49:18+00:00,NaT,eedd14d0-678c-11ea-ba7a-6b7d0b873a3d,None,2,Procter & Gamble,False,Product,23,BEAUTY,BEAUTY,ee0a3470-678c-11ea-ba7a-6b7d0b873a3d,44.0,Face / Body Care,Beauty & Health,1,53.0,1126463b-3db3-44a6-a2a6-67689d35a5b1,{},NaN,[30],[],[667],[],312.0,Beauty Other,jnjanz,2021-11-23,GCP


In [47]:
sql_reg = """select * 
        from
        (select
                event_name as recognition_event_name,
                timestamp as recognition_event_timestamp, 
                wave_type as recognition_wave_type,
                project_name,
                probe_id,
                probe_creation_time, 
                data_subject,
                if(data_subject = 'Price', true, false) as is_price_recognition_probe,
                e.tag_source_group,    
                e.tag_source,
                e.product_fk as product_pk,
                e.facings,--not same with sku level
                e.total_price_tags,
                e.price_value,
                e.is_promotion,
                dense_rank() over(partition by event_name, project_name, probe_id, e.tag_source_group order by timestamp desc) as rownum
            from `trax-ortal-prod.raw.factory_recognition`
            cross join unnest(additional_event_data) as e
            where 1=1
                and date(timestamp) > '2021-10-01'
                and wave_type in ('primary' , 'offline_pricing', 'category_expert')
                and e.tag_source_group in ('Engine', 'QAT') 
                and data_subject = 'Price'
                and flavor = 'default'
        ) 
        where 1=1
        and rownum = 1"""

In [48]:
df_recognition = bq_client.run_query(sql_reg).to_dataframe()

2021-11-24 11:36:34,763 - Diageo Accuracy Test - 43845 - INFO - Run query with job_id=fd9365db-f205-495e-993e-8a66abce5c29 
2021-11-24 11:36:56,728 - Diageo Accuracy Test - 43845 - INFO - Query job=fd9365db-f205-495e-993e-8a66abce5c29 finished {'total_mb_billed': 24792.530944, 'total_mb_processed': 24791.933245}


In [49]:
df_recognition.head()

,recognition_event_name,recognition_event_timestamp,recognition_wave_type,project_name,probe_id,probe_creation_time,data_subject,is_price_recognition_probe,tag_source_group,tag_source,product_pk,facings,total_price_tags,price_value,is_promotion,rownum
0,PROBE_RECOGNITION-DATA-SUMMARIZED,2021-10-15 13:42:52.745648+00:00,offline_pricing,diageogh,7948698,2021-10-15 12:39:50+00:00,Price,True,QAT,QAT,NaN,2,1,98.00,None,1
1,PROBE_RECOGNITION-DATA-SUMMARIZED,2021-11-17 15:39:26.536838+00:00,offline_pricing,diageogh,8033146,2021-11-17 12:46:46+00:00,Price,True,QAT,QAT,NaN,1,1,19.00,None,1
2,PROBE_RECOGNITION-DATA-SUMMARIZED,2021-11-24 08:54:42.228325+00:00,offline_pricing,diageogh,8043579,2021-11-23 14:30:45+00:00,Price,True,QAT,QAT,639.0,1,1,266.99,None,1
3,PROBE_RECOGNITION-DATA-SUMMARIZED,2021-10-26 00:13:56.094142+00:00,offline_pricing,diageoar,986210,2021-10-25 16:41:45+00:00,Price,True,Engine,Engine,173.0,1,1,730.00,None,1
4,PROBE_RECOGNITION-DATA-SUMMARIZED,2021-11-06 04:03:43.822353+00:00,offline_pricing,diageoar,991473,2021-11-05 13:14:48+00:00,Price,True,QAT,QAT,43.0,1,1,2975.76,None,1


In [52]:
df_recognition[(df_recognition['probe_id']==664187) & (df_recognition['project_name']=='jnjanz')&(df_recognition['product_pk']==5510.0)]

,recognition_event_name,recognition_event_timestamp,recognition_wave_type,project_name,probe_id,probe_creation_time,data_subject,is_price_recognition_probe,tag_source_group,tag_source,product_pk,facings,total_price_tags,price_value,is_promotion,rownum
4299131,PROBE_RECOGNITION-DATA-SUMMARIZED,2021-10-29 10:04:28.575145+00:00,offline_pricing,jnjanz,664187,2021-10-28 23:29:15+00:00,Price,True,Engine,Engine,5510.0,1,1,48.99,None,1
